# Notebook To Create MS Word Docs For Notebooks in a Folder

This notebook contains a minimum viable recipe for creating a single Microsoft Word document from the notebooks contained within a specified folder.

The recipe:

- generates a list of notebooks in a folder
- creates a single notebook from those notebooks
- writes a temporary html export of that merged notebook
- converts the html export of the merged notebooks to a docx file.

Put this notebook in the parent directory of the directory you want to process.

In [ ]:
#Function to merge notebooks
#This could be extended to add separators between, or notebook title/headers for, each notebook
# We could also extend it to allow for notebooks to be executed before being concatenated

#https://gist.github.com/fperez/e2bbc0a208e82e450f69
import io
import os
import sys

import nbformat

def merge_notebooks(filenames):
    merged = None
    for fname in filenames:
        with io.open(fname, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, as_version=4)
        if merged is None:
            merged = nb
        else:
            # TODO: add an optional marker between joined notebooks
            # like an horizontal rule, for example, or some other arbitrary
            # (user specified) markdown cell)
            merged.cells.extend(nb.cells)
    if not hasattr(merged.metadata, 'name'):
        merged.metadata.name = ''
    merged.metadata.name += "_merged"
    return nbformat.writes(merged)

In [3]:
from nbformat.v4.convert import random_cell_id
random_cell_id()

'808c51a9'

In [ ]:
#Folder you want to process
nbdir='Part 01 Notebooks'

In [ ]:
#Note - we could make a function of this
#and then iterate through all the child folders with Notebooks in the name, creating a Word doc for each
#We could even try to create a single monolithic document from notebooks in all child Notebooks folders
fn_out='{}_merged'.format(nbdir.replace(' ','_'))
with open('{}.ipynb'.format(fn_out),'w') as f:
    f.write(merge_notebooks( ['{}/{}'.format(nbdir,nb) for nb in os.listdir(nbdir) if nb.endswith('.ipynb') ]))
    
    #By default, we will just export notebooks as they are, but we can also run them prior to export.
    #However, any state set early on in a merged notebook will propagate through the other merged notebooks' code
    #Also, pandas runs dataframes in memory so processing lots of merged notebooks could be dangerous!
    #!jupyter nbconvert --to notebook --execute "$fn_out".ipynb
    #The executed notebook will be named "$fn_out".nbformat.ipynb
    #To execute in place (i.e. retaining the original filename)
    #!jupyter nbconvert --to notebook --execute "$fn_out".ipynb --output "$fn_out".ipynb
    #To execute notebooks to completion even if errors are encountered, add the flag: --allow-errors 
    
    #convert the merged document to html
    !jupyter nbconvert --to html "$fn_out".ipynb
    
    # Convert to MS Word .docx
    !pandoc -s "$fn_out".html -o "$fn_out".docx
    !rm "$fn_out".html
!rm "$fn_out".ipynb

If everything works, you should now find a Word document titled `"$fn_out".docx` in the current directory.